<a href="https://colab.research.google.com/github/mahmoud99-cell/Creating-360-deg-Views-From-Cameras-Viewing-the-Road-Scene/blob/main/opencv_python_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install --upgrade pip
!pip -V
!pip install scikit-build
!git clone https://github.com/opencv/opencv-python
!cd opencv-python && ENABLE_CONTRIB=1 python setup.py bdist_wheel -- -DWITH_CUDA=ON -DENABLE_FAST_MATH=1 -DCUDA_FAST_MATH=1 -DWITH_CUBLAS=1 -DCUDA_ARCH_BIN='6.1 7.5' -- -j8


     |████████████████████████████████| 1.6 MB 5.2 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
pip 21.2.4 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
     |████████████████████████████████| 73 kB 1.8 MB/s 
Cloning into 'opencv-python'...
remote: Enumerating objects: 2856, done.
remote: Counting objects: 100% (340/340), done.
remote: Compressing objects: 100% (183/183), done.
remote: Total 2856 (delta 150), reused 250 (delta 113), pack-reused 2516
Receiving objects: 100% (2856/2856), 1.67 MiB | 12.21 MiB/s, done.
Resolving deltas: 100% (1711/1711), done.


--------------------------------------------------------------------------------
-- Trying "Ninja" generator
--------------------------------
---------------------------
----------------------
-----------------
------------
-------
--
Not searching for unused variables given on the command line.
CMake Error: CMake 

In [ ]:
!pip install opencv-python/dist/opencv_contrib_python-4.5.3+c1cc7e5-cp37-cp37m-linux_x86_64.whl

Processing ./opencv-python/dist/opencv_contrib_python-4.5.3+c1cc7e5-cp37-cp37m-linux_x86_64.whl
  Attempting uninstall: opencv-contrib-python
    Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


In [ ]:
import cv2
cv2.__version__

!nvidia-smi

Sat Aug 21 15:29:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!wget https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov4-tiny.cfg
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights
!wget https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/coco.names
!wget https://github.com/Asadullah-Dal17/yolov4-opencv-python/raw/master/output.avi

!sed -i 's/width=416/width=1280/g' yolov4-tiny.cfg
!sed -i 's/height=416/height=736/g' yolov4-tiny.cfg

--2021-08-21 15:27:50--  https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov4-tiny.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3231 (3.2K) [text/plain]
Saving to: ‘yolov4-tiny.cfg’

yolov4-tiny.cfg     100%[===================>]   3.16K  --.-KB/s    in 0s      

2021-08-21 15:27:50 (60.6 MB/s) - ‘yolov4-tiny.cfg’ saved [3231/3231]

--2021-08-21 15:27:50--  https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github-releases.githubusercontent.com/75388965/228a9c00-3ea4-11eb-8e80-28d71569f56c?X-Amz-Algorithm=AWS

In [ ]:
import cv2 as cv
import time
Conf_threshold = 0.25
NMS_threshold = 0.4
COLORS = [(0, 255, 0), (0, 0, 255), (255, 0, 0),
          (255, 255, 0), (255, 0, 255), (0, 255, 255)]

class_name = []
with open('coco.names', 'r') as f:
    class_name = [cname.strip() for cname in f.readlines()]
# print(class_name)
net = cv.dnn.readNet('yolov4-tiny.weights', 'yolov4-tiny.cfg')
net.setPreferableBackend(cv.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv.dnn.DNN_TARGET_CUDA_FP16)

model = cv.dnn_DetectionModel(net)
model.setInputParams(size=(1280, 736), scale=1/255, swapRB=True)


cap = cv.VideoCapture('output.avi')
starting_time = time.time()
frame_counter = 0
while True:
    ret, frame = cap.read()
    frame_counter += 1
    if ret == False:
        break
    classes, scores, boxes = model.detect(frame, Conf_threshold, NMS_threshold)
    for (classid, score, box) in zip(classes, scores, boxes):
        color = COLORS[int(classid) % len(COLORS)]
        label = "%s : %f" % (class_name[classid[0]], score)
        cv.rectangle(frame, box, color, 1)
        cv.putText(frame, label, (box[0], box[1]-10),
                   cv.FONT_HERSHEY_COMPLEX, 0.3, color, 1)
    endingTime = time.time() - starting_time
    fps = frame_counter/endingTime
    print(f"FPS = {fps}")
    cv.putText(frame, f'FPS: {fps}', (20, 50),
               cv.FONT_HERSHEY_COMPLEX, 0.7, (0, 255, 0), 2)
    #cv.imshow('frame', frame)
    #key = cv.waitKey(1)
    key = 0
    if key == ord('q'):
        break
cap.release()
#cv.destroyAllWindows()

FPS = 1.2118633808306585
FPS = 1.288858938559932
FPS = 1.3292025616193166
FPS = 1.350818520105858
FPS = 1.3635205434233562
FPS = 1.3728082564144022
FPS = 1.3793583493474761
FPS = 1.3776783425668018
FPS = 1.3779678451478776
FPS = 1.382156180698451
FPS = 1.3824767486925493
FPS = 1.3851237244028132
FPS = 1.38528394039468
FPS = 1.3877212117107185
FPS = 1.3898345664468945
FPS = 1.3915833021028774
FPS = 1.3907703374545508
FPS = 1.3922484754368718
FPS = 1.3934603768781308
FPS = 1.3944701099259178
FPS = 1.3956812663248679
FPS = 1.395261300097308
FPS = 1.3964022760422445
FPS = 1.39720048831849
FPS = 1.398007480431147
FPS = 1.3987668235489987
FPS = 1.3995814511937124
FPS = 1.3992616210589672
FPS = 1.3999195552881534
FPS = 1.4000212446893423
FPS = 1.3991020523593645
FPS = 1.399762034340242
FPS = 1.400421605603053
FPS = 1.4010225477585008
FPS = 1.4014608619225488
FPS = 1.4019383607412175
FPS = 1.4024086482990579
FPS = 1.402867069662601
FPS = 1.4032246624899354
FPS = 1.4035045333901075
FPS = 1.4038

KeyboardInterrupt: ignored